In [ ]:
from astropy.io import fits, ascii
from astropy.table import Table
import numpy as np
import yaml

from pathlib import Path
import glob
import os
import configparser
import random

import constants

In [ ]:
VEGA_TEMPLATE_DIR = Path('/global/homes/b/bzh/clamato-xcorr/vega_cfg')


# Read bin edges
PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')

PiBins0 = ascii.read(PiBin_fil)['pi_edges'].data
SigBins0 = ascii.read(SigBin_fil)['sigma_edges'].data

# vega interprets these as center values.
RP_VALS = (PiBins0[1:] + PiBins0[:-1]) / 2
RT_VALS =  (SigBins0[1:] + SigBins0[:-1]) / 2

RT_GRID, RP_GRID = np.meshgrid(RT_VALS, RP_VALS)

Z_EFF = 2.3

In [ ]:
def gen_vega_config_and_data(survey_name,
                             suffix,
                             use_raw_covar=True,
                             xcorr_base=Path(constants.XCORR_DIR_BASE) / 'split',
                             output_base=Path(constants.MCMC_DIR_BASE) / '..' / 'split-mcmc'):
    output_base = output_base / survey_name
    os.makedirs(output_base, exist_ok=True)
    
    # Generate picca-format FITS file.
    obs_xcorr = np.load(xcorr_base / f'xcorr_{survey_name}_{suffix}_globalf_{constants.DATA_VERSION}.npy')
    covar_path = glob.glob(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f"covar_{'raw' if use_raw_covar else ''}mock*_"+survey_name+f"_{constants.DATA_VERSION}.npy"))
    assert len(covar_path) == 1
    covar = np.load(covar_path[0])
    assert obs_xcorr.dtype == np.double and covar.dtype == np.double
    
    data_table = Table([covar, obs_xcorr.flatten(), RT_GRID.flatten(order='F'), RP_GRID.flatten(order='F'), np.full(obs_xcorr.size, Z_EFF)],
                      names=['CO', 'DA', 'RT', 'RP', 'Z'])
    data_hdu = fits.BinTableHDU(data=data_table)
    data_hdu.header['RPMIN'] = min(RP_VALS)
    data_hdu.header['RPMAX'] = max(RP_VALS)
    data_hdu.header['NP'] = len(RP_VALS)
    data_hdu.header['RTMIN'] = min(RT_VALS)
    data_hdu.header['RTMAX'] = max(RT_VALS)
    data_hdu.header['NT'] = len(RT_VALS)
    data_hdu.header['BLINDING'] = 'none'
    hdul = fits.HDUList([fits.PrimaryHDU(), data_hdu])
    hdul_path = output_base / f'{survey_name}_{suffix}.fits'
    hdul.writeto(hdul_path, overwrite=True)
    
    # Copy template .ini files to output base folder, and change filenames.
    tracer_cfg = configparser.ConfigParser()
    tracer_cfg.optionxform=str
    tracer_cfg.read(VEGA_TEMPLATE_DIR / 'qsoxlya_template.ini')
    tracer_cfg['data']['filename'] = str(hdul_path)
    tracer_cfg_path = output_base / f'qsoxlya_{survey_name}_{suffix}.ini'
    with open(tracer_cfg_path, 'w') as f:
        tracer_cfg.write(f)
    
    main_cfg = configparser.ConfigParser()
    main_cfg.optionxform=str
    main_cfg.read(VEGA_TEMPLATE_DIR / 'main_template.ini')
    main_cfg['data sets']['ini files'] = str(tracer_cfg_path)
    with open(output_base / f'main_{survey_name}_{suffix}.ini', 'w') as f:
        main_cfg.write(f)

In [ ]:
for bin_title in constants.STACKED_BIN_TITLES:
    for s in ['vuds', 'zDeep', 'mosdef', 'clamato']:
        gen_vega_config_and_data(s, bin_title)